In [123]:
import PyPDF2
import re
import pandas as pd
import os

In [124]:
def generate_text_from_pdf(path):
    convert_text_name=path[:-4]+'.txt'
    os.system('pdftotext -layout %s %s'%(path,convert_text_name))
    return convert_text_name

在fp.readlines()中返回的是个list，每个list中的选项是byte类型的obj,pattern.search的对象必须是str，所以才加上line.decode('utf-8')

In [125]:
def extract_info_from_convert_text(path):
    pattern=re.compile(r'^\d{2}\/\d{2}\/\d{4}.*')
    info_list=[]
    with open(path,'rb') as fp:
        for line in fp.readlines():
            match=pattern.search(line.decode('utf-8'))
            if match:
                info_list.append(line.decode('utf-8'))
    return info_list

In [126]:
print(extract_info_from_convert_text('canberra_land_sales_result.txt'))

['03/02/2017      Residential             3    Throsby       1 to 20        43       8,408            20           $7,899,900             Delta Design and Construction\n', '03/02/2017      Residential             5    Throsby    12;15;17 &        44       4,667            13           $4,810,000             Roman Development Holdings\n', '03/02/2017      Residential             6    Throsby       2 to 25        45       9,865            24           $9,393,600             Delta Design and Construction\n', '03/02/2017      Residential             9    Throsby    3;6;7;9 & 11      35       2,025             5           $1,854,000                     MSL Projects\n', '03/02/2017      Residential         11       Throsby                      33       2,954             7           $2,698,600                     MSL Projects\n', '03/02/2017      Residential         15       Throsby    1-14; 1-15; 1   4; 2; 3    18,101           45           $14,631,150             Wellington Property Group\n

In [127]:
def extract_date_type_suburb_price_company_from_list(info_list):
    for item in info_list:
        date=item[:10]
        pattern_type=re.compile(r'(?:\d{2}\/\d{2}\/\d{4} +)([a-zA-Z]+)')
        block_type=pattern_type.search(item)
        pattern_price=re.compile(r'\$[0-9,]+')
        price=pattern_price.search(item)
        pattern_suburb=re.compile(r'(?<= {3})[a-zA-Z]+(?= {3})')
        
        info_dict={}
        info_dict['date']=date
        if block_type:
            info_dict['block type']=block_type.groups()[0]
            block_type_position=block_type.end()
            suburb=pattern_suburb.search(item[block_type_position+1:])
            if suburb:
                info_dict['suburb']=suburb.group()
            else:
                info_dict['suburb']=''
        else:
            info_dict['block type']=''
            info_dict['suburb']=''
        if price:
            info_dict['price']=price.group()
            purchaser_offset=price.end()
            purchaser=item[purchaser_offset+1:]
            info_dict['purchaser']=purchaser
        else:
            info_dict['price']=''
            info_dict['purchaser']=item[-10:]
                      
        print(info_dict)

In [128]:
info_list=extract_info_from_convert_text('canberra_land_sales_result.txt')
block_type_list=extract_date_type_suburb_price_company_from_list(info_list)

{'price': '$7,899,900', 'block type': 'Residential', 'date': '03/02/2017', 'purchaser': '            Delta Design and Construction\n', 'suburb': 'Throsby'}
{'price': '$4,810,000', 'block type': 'Residential', 'date': '03/02/2017', 'purchaser': '            Roman Development Holdings\n', 'suburb': 'Throsby'}
{'price': '$9,393,600', 'block type': 'Residential', 'date': '03/02/2017', 'purchaser': '            Delta Design and Construction\n', 'suburb': 'Throsby'}
{'price': '$1,854,000', 'block type': 'Residential', 'date': '03/02/2017', 'purchaser': '                    MSL Projects\n', 'suburb': 'Throsby'}
{'price': '$2,698,600', 'block type': 'Residential', 'date': '03/02/2017', 'purchaser': '                    MSL Projects\n', 'suburb': 'Throsby'}
{'price': '$14,631,150', 'block type': 'Residential', 'date': '03/02/2017', 'purchaser': '            Wellington Property Group\n', 'suburb': 'Throsby'}
{'price': '$2,800,000', 'block type': 'Industrial', 'date': '22/12/2016', 'purchaser': '

In [129]:
#block_type=list(block_type_list)
#print(block_type)

def extract_dict_from_info_list(info_list):
    dict={}
    pattern=re.compile(r'(?:[a-zA-Z0-9$,/]+ ?)+') 
    for item in info_list:
        for match in pattern.finditer(item):
            print(match.start(),match.group())

info_list=extract_info_from_convert_text('canberra_land_sales_result.txt')
extract_dict_from_info_list(info_list)